# 问题：

    1、微博的标题中有很多词汇，需要判别哪些词性的词汇最能够表示这个微博的意思。（不然引入噪声太大）
    2、微博中有很多新词汇，旧有的字典无法识别，可能需要添加自定义词库。

# 加载数据

In [17]:
import pandas as pd
import jieba as jb
import jieba.posseg as pseg
import re
import numpy as np

In [63]:
'''
加载停用词词典
'''
stop_words=[]
f=open('stop_words.txt','r',encoding='utf-8')
for li in f.readlines():
    li=li.replace('\n','')
    stop_words.append(li)
f.close()

In [64]:
'''
加载自定义词典
处理自定义词
'''
user_list=[]
with open('user_dict.txt','r',encoding='utf-8') as f:
    line=f.readline()
    while line:
        temp_word=line[:-1].split(' ')
        word_freq=None
        word_attr=None
        for aa in temp_word[1:]:
            if str.isdigit(aa):
                word_freq=aa
            elif len(aa)>0:
                word_attr=aa
        user_list.append((temp_word[0],word_freq,word_attr))
        line=f.readline()
user_list
for word in user_list:
    t_word=word[0]
    t_freq=word[1]
    t_attr=word[2]
    jb.add_word(t_word,t_freq,t_attr)

In [29]:
clear_data=pd.read_csv('clear_data.csv')

In [30]:
clear_data.head()

,comment,create_time,like,locate,repost,title,tools,like_nor,repost_nor,comment_nor,fuhao,huati1,huati2,biaoqing,cut_words
0,3939,2015-08-17 20:41,3197,无,7019,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.014482,0.059186,0.040136,.... .... ....,NaN,NaN,NaN,BBC 推送 点 25 分 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 四面佛 BBC...
1,511,2015-08-17 21:38,747,无,1097,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.003384,0.009250,0.005207,.... .... ....,NaN,NaN,NaN,BBC 推送 点 25 分 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 四面佛 BBC...
2,542,2015-08-17 22:07,2846,无,1328,话说。最近几年来，各种保守老旧的名字已经不能满足英国人了.... 最近英国公布了一份新生儿姓...,微博 weibo.com,0.012892,0.011198,0.005523,.... ... ... ... .......,NaN,NaN,NaN,话 说 保守 老旧 名字 英国人 英国 公布 一份 新生儿 姓名 调查报告 权力 游戏 里 ...
3,723,2015-08-17 23:25,4774,无,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,微博 weibo.com,0.021626,0.005026,0.007367,...... ....,【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反差？】,NaN,NaN,话 说 有个 妹纸 做 实验 素颜 妹纸 美图 前 美图 约会 软件 男人 反差 数量 整整...
4,96,2015-08-17 23:59,828,无,244,【体验-伦敦】谁说看电影只能啃爆米花的？号称电影院里伙食最好的Edible Cinema把自...,街边捡来的iPhone 6,0.003751,0.002057,0.000978,NaN,【体验-伦敦】,NaN,NaN,体验 伦敦 说 电影 只能 啃 爆米花 号称 电影院 里 伙食 EdibleCinema 自...


In [31]:
test_dic={}
test_dic.keys().__contains__('a')

False

# 分词&去除停用词

    1、这里要注意顺序，先进行分词，再进行停用词去除
    2、这里最终得到了一个每个词出现的次数，以及分词后的句子

In [65]:
# 根据词性进行分词
word_dict={}
target_word=['v','ns','n','v']
rest_word=[]
for i,raw_title in enumerate(clear_data.title):
    temp_words=pseg.cut(raw_title)
    temp_list=[]
    for word,flag in temp_words:
        if flag in target_word and word not in stop_words and len(word)>1:
            temp_num=word_dict.get(word,0)
            word_dict[word]=temp_num+1
            temp_list.append(word)
    rest_word.append(' '.join(temp_list))

In [67]:
clear_data['cut_words']=rest_word
clear_data.head()

,comment,create_time,like,locate,repost,title,tools,like_nor,repost_nor,comment_nor,fuhao,huati1,huati2,biaoqing,cut_words,word_bag
0,3939,2015-08-17 20:41,3197,无,7019,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.014482,0.059186,0.040136,.... .... ....,NaN,NaN,NaN,推送 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 记者 现场 伤亡,"{0: 1, 1: 1, 2: 1, 3: 2, 4: 1, 5: 1, 6: 1, 7: ..."
1,511,2015-08-17 21:38,747,无,1097,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.003384,0.009250,0.005207,.... .... ....,NaN,NaN,NaN,推送 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 记者 现场 伤亡,"{0: 1, 1: 1, 2: 1, 3: 2, 4: 1, 5: 1, 6: 1, 7: ..."
2,542,2015-08-17 22:07,2846,无,1328,话说。最近几年来，各种保守老旧的名字已经不能满足英国人了.... 最近英国公布了一份新生儿姓...,微博 weibo.com,0.012892,0.011198,0.005523,.... ... ... ... .......,NaN,NaN,NaN,保守 名字 英国 公布 新生儿 姓名 调查报告 权力 游戏 人物 起名,"{10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16:..."
3,723,2015-08-17 23:25,4774,无,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,微博 weibo.com,0.021626,0.005026,0.007367,...... ....,【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反差？】,NaN,NaN,妹纸 素颜 妹纸 约会 软件 男人 反差 数量 感受,"{21: 2, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27:..."
4,96,2015-08-17 23:59,828,无,244,【体验-伦敦】谁说看电影只能啃爆米花的？号称电影院里伙食最好的Edible Cinema把自...,街边捡来的iPhone 6,0.003751,0.002057,0.000978,NaN,【体验-伦敦】,NaN,NaN,体验 伦敦 电影 只能 爆米花 号称 电影院 伙食 自制 美食 观影 体验 小吃 精心 号码...,"{29: 3, 30: 1, 31: 2, 32: 1, 33: 1, 34: 1, 35:..."


In [18]:
# 词典
word_dict={}

#分词，并去除停用词
for i,raw_title in enumerate(clear_data.title):
    # 先把title去空格
    raw_title=''.join(raw_title.split())
    # 分词
    title_list=jb.cut(raw_title)
    # 去停用词
    final_list=[]
    for ttl in title_list:
        if ttl not in stop_words:
            final_list.append(ttl)
    
    for final_word in set(final_list):
        
        temp_num=word_dict.get(final_word,0)
        word_dict[final_word]=temp_num+1
        

    
    clear_data.loc[i,'cut_words']=' '.join(final_list)

In [13]:
clear_data.head()

,comment,create_time,like,locate,repost,title,tools,like_nor,repost_nor,comment_nor,fuhao,huati1,huati2,biaoqing,cut_words
0,3939,2015-08-17 20:41,3197,无,7019,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.014482,0.059186,0.040136,.... .... ....,NaN,NaN,NaN,BBC 推送 点 25 分 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 四面佛 BBC...
1,511,2015-08-17 21:38,747,无,1097,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.003384,0.009250,0.005207,.... .... ....,NaN,NaN,NaN,BBC 推送 点 25 分 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 四面佛 BBC...
2,542,2015-08-17 22:07,2846,无,1328,话说。最近几年来，各种保守老旧的名字已经不能满足英国人了.... 最近英国公布了一份新生儿姓...,微博 weibo.com,0.012892,0.011198,0.005523,.... ... ... ... .......,NaN,NaN,NaN,话 说 保守 老旧 名字 英国人 英国 公布 一份 新生儿 姓名 调查报告 权力 游戏 里 ...
3,723,2015-08-17 23:25,4774,无,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,微博 weibo.com,0.021626,0.005026,0.007367,...... ....,【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反差？】,NaN,NaN,话 说 有个 妹纸 做 实验 素颜 妹纸 美图 前 美图 约会 软件 男人 反差 数量 整整...
4,96,2015-08-17 23:59,828,无,244,【体验-伦敦】谁说看电影只能啃爆米花的？号称电影院里伙食最好的Edible Cinema把自...,街边捡来的iPhone 6,0.003751,0.002057,0.000978,NaN,【体验-伦敦】,NaN,NaN,体验 伦敦 说 电影 只能 啃 爆米花 号称 电影院 里 伙食 EdibleCinema 自...


In [50]:
jb.cut??

In [68]:
temp_str=clear_data.loc[394,'title']
aaa=pseg.cut(temp_str)
for word,flag in aaa:
    print(word,flag)

人家 n
就是 d
那么 r
一直 d
会 v
卖萌 v
的 uj
傻白甜 x
...... m
[ x
害羞 a
] x
[ x
害羞 a
] x
[ x
害羞 a
] x
  x


# 拿到list的词典，并转换词袋模型

In [69]:
word_li=list(word_dict.keys())
for i,c_words in enumerate(clear_data.cut_words):
    # 对分好的词进行逐一统计
    word_list=c_words.split(' ')
    temp_dic={}
    print(c_words,i)
    for word in word_list:
        # 拿到每个词在字典中的index
        temp_index=word_li.index(word)
        # 统计个数
        if temp_index in temp_dic.keys():
            temp_dic[temp_index]+=1
        else:
            temp_dic[temp_index]=1
    clear_data.loc[i,'word_bag']=str(temp_dic)

推送 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 记者 现场 伤亡 0
推送 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 记者 现场 伤亡 1
保守 名字 英国 公布 新生儿 姓名 调查报告 权力 游戏 人物 起名 2
妹纸 素颜 妹纸 约会 软件 男人 反差 数量 感受 3
体验 伦敦 电影 只能 爆米花 号称 电影院 伙食 自制 美食 观影 体验 小吃 精心 号码 让你在 电影 精彩 时刻 享受 味蕾 视觉 双重 享受 放映 想去 介绍 体验 4
世界 年龄 四胞胎 母亲 男人 孩子 5
吸血 虎牙 天生 主人 起名 恶煞 对得起 名字 6
有没有 西瓜 哥哥 餐厅 请到 迎接 主人 回家 可爱 精彩 节目 好玩 游戏 环节 等你拿 回家 爱心 转发 7
扮演 蝙蝠侠 探望 生病 小朋友 视频 响应 团体 邀请 小朋友 带来 勇气 高速公路 故障 蝙蝠 身亡 英雄 蜡烛 8
穿着 内衣 伦敦 广场 爱惜 身体 事情 9
鼓励 读书 公交车 免票 10
流浪狗 喂成 大肥 瘦身 美女 主人 11
学业成绩 理想 担心 机会 英国 名校 学习 学习 集团 提供 包含 预科 国际 硕士 预科 课程 通过率 拥有 超过 英国 大学 道路 提供 机会 12
调戏 关键时刻 小哥 13
吃货 妹子 可爱 早餐 小伙伴 时间 精力 早饭 早饭 吃到 早餐 份量 文章 可爱 早餐 吃货 14
假装 人工 翻译 关注 带上 话题 假装 发布 假装 图片 文字 抽取 自助游 假装 帆船 酒店 早餐 假装 度假 自娱自乐 下图 15
视觉 错觉 小哥 合辑 16
北极熊 加拿大 摄影师 拍下 改玩 反差 17
公举 女儿 透透 手心 天际 控大 布布 18
庆祝 入驻 中国 寺库 奢侈品 网站 上线 皇家 中国 享有 英国 包袋 无需 等待 关税 考考 寺库 价格 提示 寺库 品牌 奢侈品 大牌 转发 置顶 写出 答案 19
工作犬 法庭 安慰 出庭作证 证人 目睹 事件 证人 心灵 20
工作犬 法庭 安慰 出庭作证 证人 目睹 事件 证人 心灵 21
英国 中学 请来 中国 老师 经历 英国 学生 叛逆 老师 展示 因材施教 补课 实心球 国旗 讲话 扇子舞 中国 全胜 内容 视频 审核 文字 整理 22
英国 外遇 外遇 网站 下载 网站 数据库 网站 关闭 公布 

纹身 计划 受过 暴力 女性 纹身 遮盖 伤疤 重拾 自信 小事 保护 女性 远离 暴力 228
中国 大使 舌战 阅兵 英国 大使 接受 新闻节目 新闻 专访 节目 中国 威胁论 中国 军费 中国 言论 股市 大跌 大使 回应 视频 229
难民 背包 战争 洗刷 幻想 留下 渴望 背包 缩影 国际 委员会 采访 难民 行囊 探寻 230
英国 夫妇 宠物 嫉妒 231
发射 火箭 地面 民众 拍到 场面 全程 加速 232
记得 爆炸 歌手 外文 电音 咖尚 阅兵 文化交流 大使 发现 参加 中法 国宴 麻麻 过手 会见 加拿大 总理 主席 挪威 变身 外交官 同学 娱乐圈 233
伦敦 摄影师 公布 外派 摄影师 女王 照片 常规 见到 新闻 皇宫 私底下 女王 234
犯人 统治 监狱 国家 犯人 狱警 踏入 伦敦 摄影师 小哥 片子 235
听说 英国 超市 国际 开店 熟悉 英国 坚果 英国 日用品 母婴 用品 英国 带来 236
狗狗 主人 事情 237
小猫 玩具 小女孩 玩具 袋鼠 238
快餐 岛国 不出 239
德国 伦敦 展开 空袭 伦敦 轰炸 超过 超过 空袭 死亡 房屋 摧毁 英国 媒体 整理 空袭 240
母女 妈妈 女儿 后花园 玩耍 组合 女儿 名山大川 241
体验 伦敦 优质 餐厅 聚集地 推荐 超赞 意大利 主厨 月份 回归 执掌 厨房 食物 摆盘 味道 积攒 口碑 时期 团购 道菜 起泡 团购 242
学校 开学 坦克 孩子 上学 驻军 坦克 指挥官 欺负 243
演员 合约 演员 媒体 有意 出演 粉丝 都乐 喜欢 画风 244
英国 烹饪 垃圾 食物 饭店 意义 245
女王 在位 君主 打算 仪式 打算 低调 女王 打破 记录 打破 记录 246
英国 移民 政策 汇总 留学 英国 优姐 专业 资深 移民 律师 超全 干货 评论 英国 留学 英国 247
大洋 对岸 毕业生 经历 传奇 学位 轰炸机 时代周刊 年度 总统 哥们 现实 说起 248
英国 旅馆 酒店 英国 运河 发达 不想 小船 房船 快递 249
患有 重度 拥抱 渴望 袋熊 抱紧 250
提出 英国 接受 难民 英国 网民 心累 舆论 推着 网友 评论 251
不让 酒店 英国 夫妇 改装 房车 要住 改装 双层 巴士 252
伦敦 健身 白费 盼来

心累 种族 英国 受尽 歧视 欺负 警察 自首 希望 遣返 警察 想骗 回国 机票 446
安利 上订 实用 贴心 提醒 居家 八卦 必备 佳品 转需 447
退学 学生 买来 手枪 子弹 催泪瓦斯 炸弹 宣扬 要炸 学校 警方 抓获 搜出 武器库 448
伦敦 小吃 小吃 接地 吃货 味道 不输给 餐馆 碰上 解馋 管饱 伦敦 小吃 伦敦 小吃 449
梦想 辞职 夫妻 梦想 开了个 450
摄影师 照片 拼出 魔性 动图 451
抱歉 作者 查看 权限 查看 452
黑镜 出新 黑镜 原班 人马 签约 开拍 收编 集爽 并卵 预告片 视频 453
入职 警犬 上班 454
德国 围城 开放 接纳 难民 风光 面对 开门 景象 大街 经济 移民 陷入 困局 原本 巴基斯坦 国家 体面 德国 只能 刷刷 盘子 难民营 女眷 提防 强奸 455
距离 小时 飞机 火车 时差 见面 提前 机票 使馆 护照 爱情 只能 想念 拥抱 爱情 经历 异国恋 恋人 接受 采访 感情 催泪 异国恋 异地 恋能 456
警方 状况 奸杀 谋杀 受害人 重现 人间 457
保守 变装 鸡奸 论处 文章 领略 变装 皇后 风潮 变装 大胆 举动 带入 社会 458
中国 球迷 关注 英国 足球 座谈会 伦敦 关乎 中国 足球 中学生 足球 发现 校园 足球 项目 孩子 球童 赛场 总理 女士 中国 足球 459
姑娘 家传 世纪 婚纱 穿过 穿过 妈妈 穿过 大姨 二姨 穿过 姑娘 结婚 婚纱 交到 家传 婚纱 掌门人 460
新技能 欧洲 驴友 打造 体验 传统 方式 刷新 五感 体验 解锁 推荐 关注 转发 好友 机会 现金 平台 欧洲 体验 平台 461
英国 老人 老奶奶 拿出 退休金 凑钱 请来 感觉 现场 462
国家 地理 摄影师 镜头 火山 岩浆 纹路 视频 视频 463
入职 警察 要换 路线 巡逻 救下 自杀 小女孩 464
小男孩 描述 初恋 465
丸子 神马 哥们 玩起 麻花 466
视频 467
看完 视频 发现 翼装 飞行 接近 超人 视频 468
热议 校园 暴力 漫画 讲述 遭受 校园 暴力 丑女 整容 回校 复仇 故事 连载 469
姑娘 女子 小队 对抗 消灭 分子 470
变色龙 卵胎生 妈妈 孩子 变色龙 过程 妈妈 孩子 妈妈 出生 孩子 妈妈

ValueError: '' is not in list

In [44]:
clear_data.head()

,comment,create_time,like,locate,repost,title,tools,like_nor,repost_nor,comment_nor,fuhao,huati1,huati2,biaoqing,cut_words,word_bag
0,3939,2015-08-17 20:41,3197,无,7019,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.014482,0.059186,0.040136,.... .... ....,NaN,NaN,NaN,推送 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 记者 现场 伤亡,"{0: 1, 1: 1, 2: 1, 3: 2, 4: 1, 5: 1, 6: 1, 7: ..."
1,511,2015-08-17 21:38,747,无,1097,BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.....,微博 weibo.com,0.003384,0.009250,0.005207,.... .... ....,NaN,NaN,NaN,推送 泰国 曼谷 市中心 发生 爆炸 市中心 旅游景点 记者 现场 伤亡,"{0: 1, 1: 1, 2: 1, 3: 2, 4: 1, 5: 1, 6: 1, 7: ..."
2,542,2015-08-17 22:07,2846,无,1328,话说。最近几年来，各种保守老旧的名字已经不能满足英国人了.... 最近英国公布了一份新生儿姓...,微博 weibo.com,0.012892,0.011198,0.005523,.... ... ... ... .......,NaN,NaN,NaN,保守 名字 英国 公布 新生儿 姓名 调查报告 权力 游戏 人物 起名,"{10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16:..."
3,723,2015-08-17 23:25,4774,无,596,话说，有个妹纸最近做了个实验：【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反...,微博 weibo.com,0.021626,0.005026,0.007367,...... ....,【素颜的妹纸，美图前和美图后，约会软件上的男人们都会有怎样的反差？】,NaN,NaN,妹纸 素颜 妹纸 约会 软件 男人 反差 数量 感受,"{21: 2, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27:..."
4,96,2015-08-17 23:59,828,无,244,【体验-伦敦】谁说看电影只能啃爆米花的？号称电影院里伙食最好的Edible Cinema把自...,街边捡来的iPhone 6,0.003751,0.002057,0.000978,NaN,【体验-伦敦】,NaN,NaN,体验 伦敦 电影 只能 爆米花 号称 电影院 伙食 自制 美食 观影 体验 小吃 精心 号码...,"{29: 3, 30: 1, 31: 2, 32: 1, 33: 1, 34: 1, 35:..."


# 词袋模型转TF-IDF模型
 TF 计算：
 
     某个词出现的频率/这篇文章所有词的总数
 
 IDF 计算：
 
     log（文档总数/[出现该词的文档数+1]）

In [17]:
test=eval(clear_data.word_bag[0])
all_res=np.sum(list(test.values()))
all_res

17

In [18]:
test

{5878: 2,
 8585: 1,
 11535: 1,
 13749: 1,
 15154: 1,
 15485: 1,
 18848: 1,
 22118: 1,
 22400: 1,
 25007: 1,
 25060: 1,
 28543: 2,
 32279: 1,
 34225: 1,
 36185: 1}

In [25]:
# 计算TF
def cal_tf(word_bag):
    # 传入词袋
    word_tfs={}
    # 拿到每个词的index
    words=word_bag.keys()
    # 计算总共出现词的个数
    total_words=np.sum(list(word_bag.values()))
    # 计算词频
    for word in words:
        cnt=int(word_bag[word])
        temp_tf=1.0*cnt/total_words
        word_tfs[word]=temp_tf
    return word_tfs

In [48]:
# 计算 IDF
def cal_idf(word_bag,word_dict,file_num):
    # 拿到词典
    word_list=list(word_dict.keys())
    # 拿到所有词
    words_ind=word_bag.keys()
    # 结果字典
    res={}
    for word_ind in words_ind:
        # 通过序号拿到真的词
        real_word=word_list[word_ind]
        # 拿到该词在哪些文档中出现过
        fre=word_dict[real_word]
        # 计算IDF
        temp=np.log(file_num/(1+fre))
        res[word_ind]=temp
    return res
    

In [53]:
# 计算总的 TF—IDF
def cal_tf_idf(word_bag,word_dict,file_num):
    tf=cal_tf(word_bag)
    idf=cal_idf(word_bag,word_dict,file_num)
    res={}
    for key in tf.keys():
        res[key]=tf[key]*idf[key]
    return res

In [49]:
test_aaa=cal_tf(test)
test_aaa

{5878: 0.11764705882352941,
 8585: 0.058823529411764705,
 11535: 0.058823529411764705,
 13749: 0.058823529411764705,
 15154: 0.058823529411764705,
 15485: 0.058823529411764705,
 18848: 0.058823529411764705,
 22118: 0.058823529411764705,
 22400: 0.058823529411764705,
 25007: 0.058823529411764705,
 25060: 0.058823529411764705,
 28543: 0.11764705882352941,
 32279: 0.058823529411764705,
 34225: 0.058823529411764705,
 36185: 0.058823529411764705}

In [51]:
test_bbb=cal_idf(test,word_dict,clear_data.shape[0])
test_bbb

{5878: 7.949356201821739,
 8585: 7.949356201821739,
 11535: 8.23703827427352,
 13749: 7.726212650507529,
 15154: 8.642503382381683,
 15485: 6.563061840701848,
 18848: 8.23703827427352,
 22118: 8.642503382381683,
 22400: 8.642503382381683,
 25007: 8.23703827427352,
 25060: 8.642503382381683,
 28543: 8.642503382381683,
 32279: 8.642503382381683,
 34225: 8.642503382381683,
 36185: 8.642503382381683}

In [55]:
test_ccc=cal_tf_idf(test,word_dict,clear_data.shape[0])
test_ccc

{5878: 0.9352183766849105,
 8585: 0.46760918834245524,
 11535: 0.48453166319255997,
 13749: 0.4544830970886782,
 15154: 0.5083825519048049,
 15485: 0.3860624612177558,
 18848: 0.48453166319255997,
 22118: 0.5083825519048049,
 22400: 0.5083825519048049,
 25007: 0.48453166319255997,
 25060: 0.5083825519048049,
 28543: 1.0167651038096097,
 32279: 0.5083825519048049,
 34225: 0.5083825519048049,
 36185: 0.5083825519048049}

In [96]:
# 文档总数
file_num=clear_data.shape[0]
for i,wd_bag in enumerate(clear_data.word_bag):
    # str转dic
    wd_bag_dic=eval(wd_bag)
    # 计算tf_idf
    temp_res=cal_tf_idf(wd_bag_dic,word_dict,file_num)
    temp_res_top_5=sorted(temp_res.items(), key=lambda d:d[1], reverse = True)
    # 保存
    clear_data.loc[i,'tf_idf']=str(temp_res)
    clear_data.loc[i,'tf_idf_top5']=str(temp_res_top_5[:5])

In [97]:
pd.set_option('display.max_colwidth',1000)
clear_data[['word_bag','tf_idf','tf_idf_top5']].head()

,word_bag,tf_idf,tf_idf_top5
0,"{128: 1, 30817: 1, 27203: 1, 30113: 1, 28457: 1, 12810: 1, 27111: 1, 6784: 1, 19698: 2, 12670: 1, 7353: 1, 12762: 1, 27322: 1, 11646: 2, 31061: 1}","{128: 0.2460211157908147, 30817: 0.2564068717953562, 27203: 0.41990741091796524, 30113: 0.2984100403470773, 28457: 0.46760918834245524, 12810: 0.24500979811196924, 27111: 0.4544830970886782, 12762: 0.30733980580990106, 19698: 0.7721249224355116, 11646: 0.5565271031710044, 7353: 0.3113293703004164, 6784: 0.4544830970886782, 27322: 0.34172294343090986, 12670: 0.48453166319255997, 31061: 0.3113293703004164}","[(19698, 0.7721249224355116), (11646, 0.5565271031710044), (12670, 0.48453166319255997), (28457, 0.46760918834245524), (27111, 0.4544830970886782)]"
1,"{128: 1, 30817: 1, 27203: 1, 30113: 1, 28457: 1, 12810: 1, 27111: 1, 6784: 1, 19698: 2, 12670: 1, 7353: 1, 12762: 1, 27322: 1, 11646: 2, 31061: 1}","{128: 0.2460211157908147, 30817: 0.2564068717953562, 27203: 0.41990741091796524, 30113: 0.2984100403470773, 28457: 0.46760918834245524, 12810: 0.24500979811196924, 27111: 0.4544830970886782, 12762: 0.30733980580990106, 19698: 0.7721249224355116, 11646: 0.5565271031710044, 7353: 0.3113293703004164, 6784: 0.4544830970886782, 27322: 0.34172294343090986, 12670: 0.48453166319255997, 31061: 0.3113293703004164}","[(19698, 0.7721249224355116), (11646, 0.5565271031710044), (12670, 0.48453166319255997), (28457, 0.46760918834245524), (27111, 0.4544830970886782)]"
2,"{33159: 1, 13320: 1, 35614: 1, 4239: 1, 33812: 1, 30677: 2, 21017: 1, 12890: 2, 6095: 1, 23644: 1, 7581: 1, 25950: 1, 16738: 1, 13350: 1, 30313: 1, 6698: 1, 8301: 1, 17072: 1, 3912: 1, 36147: 1, 14836: 1, 181: 1, 180: 1, 36474: 1, 15935: 1}","{33159: 0.28615602409287144, 13320: 0.0982793320552526, 4239: 0.12664624075311828, 33812: 0.30507549163975994, 30677: 0.35040595384841017, 25950: 0.22654721252123808, 21017: 0.17446954628360584, 12890: 0.40792660492240995, 6095: 0.19412244447109364, 23644: 0.1767149026471775, 7581: 0.14616958641204178, 35614: 0.09209571408548785, 16738: 0.28615602409287144, 13350: 0.06601347990152631, 30313: 0.2944206000674718, 6698: 0.2944206000674718, 8301: 0.2736940894031969, 17072: 0.2944206000674718, 3912: 0.19871698701442622, 36147: 0.19231540135370728, 14836: 0.095508080507188, 181: 0.20087509472272158, 180: 0.17869596505276958, 36474: 0.32009271786598825, 15935: 0.21515889030835064}","[(12890, 0.40792660492240995), (30677, 0.35040595384841017), (36474, 0.32009271786598825), (33812, 0.30507549163975994), (30313, 0.2944206000674718)]"
3,"{7745: 2, 26274: 1, 25957: 1, 1830: 1, 11207: 1, 13320: 1, 1545: 1, 15466: 1, 33935: 1, 304: 1, 23186: 1, 27028: 2, 3568: 1, 22327: 1, 4239: 1, 33034: 1, 6942: 1, 28528: 1}","{7745: 0.45234662075692117, 26274: 0.3281530920350924, 25957: 0.1594660652636366, 1830: 0.21118313872925432, 11207: 0.23456298319001287, 13320: 0.13267709827459104, 1545: 0.3078798366296842, 15466: 0.2849032201607622, 33935: 0.38631063252537645, 304: 0.1946616426209918, 23186: 0.2967226590639737, 27028: 0.6770701205480093, 3568: 0.331380018091971, 22327: 0.24894708681260191, 4239: 0.17097242501670973, 33034: 0.3425371956576815, 6942: 0.3628104510630897, 28528: 0.19031107377151033}","[(27028, 0.6770701205480093), (7745, 0.45234662075692117), (33935, 0.38631063252537645), (6942, 0.3628104510630897), (33034, 0.3425371956576815)]"
4,"{35841: 1, 12994: 1, 12939: 1, 32964: 1, 901: 1, 23366: 1, 2642: 1, 22088: 1, 20043: 1, 11980: 1, 35394: 1, 17550: 1, 29967: 2, 5906: 1, 23805: 1, 31508: 2, 981: 1, 28434: 1, 18075: 1, 23752: 1, 34721: 1, 16610: 1, 2979: 1, 18276: 1, 17574: 1, 22802: 1, 20379: 1, 22354: 1, 24110: 1, 4683: 3, 8624: 1, 13320: 1, 19378: 1, 22260: 1, 2159: 1, 33620: 1, 14836: 2, 1210: 1, 28861: 1, 5759: 1}","{35841: 0.19205563071959297, 12994: 0.06922356428537534, 20043: 0.14881318296280824, 32964: 0.116846958200798, 901: 0.183045294983856, 23366: 0.07328399206709982, 22088: 0.183045294983856, 12939: 0.1387690691018514, 11980: 0.1249350776941627, 35394: 0.16764202

In [98]:
clear_data.loc[0,['title','tf_idf_top5']]

title                                                                                          BBC推送： 7点25分左右泰国曼谷市中心发生大爆炸.... 在市中心旅游景点四面佛附近.... BBC记者表示现场伤亡严重.... 
tf_idf_top5    [(19698, 0.7721249224355116), (11646, 0.5565271031710044), (12670, 0.48453166319255997), (28457, 0.46760918834245524), (27111, 0.4544830970886782)]
Name: 0, dtype: object

In [99]:
temp_tf=eval(clear_data.loc[0,'tf_idf'])
sorted(temp_tf.items(), key=lambda d:d[1], reverse = True)

[(19698, 0.7721249224355116),
 (11646, 0.5565271031710044),
 (12670, 0.48453166319255997),
 (28457, 0.46760918834245524),
 (27111, 0.4544830970886782),
 (6784, 0.4544830970886782),
 (27203, 0.41990741091796524),
 (27322, 0.34172294343090986),
 (7353, 0.3113293703004164),
 (31061, 0.3113293703004164),
 (12762, 0.30733980580990106),
 (30113, 0.2984100403470773),
 (30817, 0.2564068717953562),
 (128, 0.2460211157908147),
 (12810, 0.24500979811196924)]

In [100]:
temp_list=list(word_dict.keys())

In [101]:
temp_list[30113]

'爆炸'

# 字典转向量
呃，之前为了存储方便，使用字典来进行词频和tf-idf的统计，现在要聚类了，只能够使用向量来操作了

In [119]:
file_list=[]
dic_len=len(word_dict)
test_set=set()
for tf_idf in clear_data.tf_idf_top5[:5]:    
    temp_li=eval(tf_idf)
    fi_li=np.zeros(dic_len)
    for tp in temp_li:
        test_set.add(tp[0])
        fi_li[tp[0]]=tp[1]
    file_list.append(list(fi_li))

print(file_list)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [112]:
clear_data[['to_list']].head()

,to_list
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [102]:
clear_data.to_csv('processed_data.csv')